In [1]:
!pip install streamlit
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirement ['pillow>=10.3.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 6.7s, installed 1 package: ['pillow>=10.3.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-9-6 Python-3.10.12 torch-2.4.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 117MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [10]:
%%writefile app.py
import numpy as np
import streamlit as st
import cv2
import torch
from PIL import Image
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
#title
st.title("Traffic Light Detection Application")

#upload image
uploaded_file = st.file_uploader("Choose an image...", type= ['jpg', 'png', 'jpeg'])

#define a function to detect colors
def Color_Detection(img, box):
  #crop image
  x1, y1, x2, y2 = map(int, box)
  cropped_img = img[y1:y2, x1:x2]
  #convert image
  hsv_img = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2HSV)
  #detect colors
  red_lower1 = np.array([0,70,50])
  red_upper1 = np.array([10,255,255])
  red_lower2 = np.array([170,70,50])
  red_upper2 = np.array([180,255,255])

  green_lower = np.array([40,40,40])
  green_upper = np.array([80,255,255])

  yellow_lower = np.array([20,100,100])
  yellow_upper = np.array([30,255,255])

  #check for red mask
  red_mask1 = cv2.inRange(hsv_img, red_lower1, red_upper1)
  red_mask2 = cv2.inRange(hsv_img, red_lower2, red_upper2)
  red_mask = red_mask1 | red_mask2
  red_pixels = cv2.countNonZero(red_mask)

  #check for green mask
  green_mask = cv2.inRange(hsv_img, green_lower, green_upper)
  green_pixels = cv2.countNonZero(green_mask)

  #check for yellow mask
  yellow_mask = cv2.inRange(hsv_img, yellow_lower, yellow_upper)
  yellow_pixels = cv2.countNonZero(yellow_mask)

  #determine color
  if max(red_pixels, green_pixels, yellow_pixels) == red_pixels:
    return "Red"
  elif max(red_pixels, green_pixels, yellow_pixels) == green_pixels:
    return "Green"
  elif max(red_pixels, green_pixels, yellow_pixels) == yellow_pixels:
    return "Yellow"

if uploaded_file is not None:
  img = Image.open(uploaded_file)
  img_array = np.array(img)
  st.image(img_array, use_column_width = True)
  #perform detection
  detection_result = model(img_array)

  #get bounding box
  boxes = detection_result.xyxy[0].numpy()

  #display detection results
  for box in boxes:
    label = detection_result.names[int(box[5])]
    if label == 'traffic light':
      color = Color_Detection(img_array, box[:4])
      st.write(f'Detected Traffic Light :, {color}')

  #show bounding box
  st.image(detection_result.render()[0], caption = 'Detected Traffic Light', use_column_width= True)

Overwriting app.py


In [4]:
#setup ngrok with your authonken
!ngrok authtoken 2JQ7xQIMC37Y473rZ6ZbapzlPMB_4cZK6X28eFyAPsEfKCwD8

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
from pyngrok import ngrok
!streamlit run app.py &>/dev/null&

In [6]:
!pgrep -f streamlit

713


In [7]:
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "https://c623-35-238-186-106.ngrok-free.app" -> "http://localhost:8501">

In [12]:
ngrok.kill()